In [1]:
import numpy as np
import pandas as pd
import torch.nn as nn
import torch as t
from torch.autograd import Variable
import torch.optim as optim
import matplotlib.pyplot as plt
import time
import re
import jieba
from collections import Counter
from openpyxl import load_workbook

from tqdm import tqdm 
 
%matplotlib inline

In [2]:
content = pd.read_excel(r"./附件1/酒店评论.xlsx",engine='openpyxl')    

label = pd.read_excel(r"./附件2/酒店评分.xlsx",engine='openpyxl')    


In [3]:

def participle(stopwords,is_filter=True,):
    all_words = []
    for idx,line in enumerate(content['评论内容']):
        if is_filter:         
            line = filter_punc(line)
       
        words = jieba.lcut(line)
        if len(words)>0:
            all_words += words
            
    dictions = {}
    cnt = Counter(all_words)
    for word,freq in cnt.items():
        if word not in stopwords:
            dictions[word] = [len(dictions),freq]
    return dictions

# 过去标点符号
def filter_punc(sentence):
    sentence = re.sub("[\s+\.\!\/_,$%^*(+\"\'“”《》?“]+|[+——！，。？、~@#￥%……&*（）：]+", "", sentence)
    return(sentence)
# 去除停用词

def move_stopwords(sentence_list, stopwords_list):
    # 去停用词
    out_list = []
    for word in sentence_list:
        if word not in stopwords_list:
#             if not remove_digits(word):
#                 continue
            if word != '\t':
                out_list.append(word)
    return out_list

#使用停用词
def stopwordslist(filepath):
    stopwords = [line.strip() for line in open(filepath, 'r', encoding='utf-8').readlines()]
    return stopwords

#得到分词后的词句，和对应的标签
def participle1(name,stopwords,is_filter=True):
    sentence = []
    labels=[]
    
    for j,i in enumerate(name):
        data = content.loc[content['酒店名称']==i]
        a=[label['总得分'][j],label['服务得分'][j],label['位置得分'][j],label['设施得分'][j],label['卫生得分'][j],label['性价比得分'][j]]
        for idx,line in enumerate(data['评论内容']):
            if is_filter:         
                line = filter_punc(line)
            
            words = jieba.lcut(line)
            words = move_stopwords(words,stopwords)
            if len(words)>0:
                sentence.append(words)
                labels.append(a) 
    return sentence,labels
    


In [4]:
names = []
stopwords = stopwordslist('hit_stopwords.txt')
for i in list(content.groupby('酒店名称')):
    names.append(i[0])  


diction = participle(stopwords,True)
sentence,labels = participle1(names,stopwords,True)


Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\RunRun\AppData\Local\Temp\jieba.cache
Loading model cost 0.700 seconds.
Prefix dict has been built successfully.


In [5]:
len(labels  )


25176

In [6]:
# 根据单词返还单词的编码
def word2index(word,diction):
    if word in diction:
        value=diction[word][0]
    else:
        value=-1
    return(value)

# 更具编码获取对应的单词
def index2word(index,diction):
    for w,v in diction.items():
        if v[0]==index:
            return(w)
    return(None)
# 输入一个句子和相应的词典，得到这个句子的向量化表示
# 向量的尺寸为词典中词汇的个数，i位置上面的数值为第i个单词在sentence中出现的频率
def sentence2vec(sentence,dictionary):
    vector = np.zeros(len(dictionary))
    for l in sentence:
        vector[l]+=1
    return vector
#     return ((1.0*vector)/len(sentence))

In [7]:
dataset=[]  #数据集
sentences=[] #原始句子
labels1=[]
j=0
for i in sentence:
    new_sentence=[]
    j+=1
    for l in i:
        if l in diction:
            new_sentence.append(word2index(l,diction))
    if len(new_sentence)!=0:
        dataset.append(sentence2vec(new_sentence,diction))
        labels1.append(labels[j-1])
    sentences.append(i)
labels=labels1

In [8]:
len(labels)

25176

In [9]:
indices=np.random.permutation(len(dataset))
dataset = [dataset[i] for i in indices]
labels = [labels[i] for i in indices]

In [10]:
test_size = len(dataset) // 10
# 训练集
train_data = dataset[2*test_size :]
train_label=labels[2*test_size :]
# 检测集
valid_data = dataset[: test_size]
valid_label = labels[: test_size]
# 测试集
test_data = dataset[test_size : 2*test_size]
test_label = labels[test_size : 2*test_size]


In [33]:
model = nn.Sequential(
    nn.Linear(len(diction),12),
    nn.ReLU(),
    nn.Linear(12,1),

)





In [35]:
# 自定义的计算一组数据分类准确度的函数
def rightness(predictions,labels):
    
    j=0
#     for i in range(6):
#         a=np.round(predictions[0][i].data,1).eq(np.round(labels[0][i].data,1))
#         if a:
#             j+=1
    a=np.round(predictions.data,1).eq(np.round(labels.data,1))
    if a:
        j+=1
    return j,1


In [31]:
 rights=[]
for i,data in enumerate(zip(train_data,train_label)):
    x,y=data
    x=Variable(t.FloatTensor(x).view(1,-1))
    y=Variable(t.FloatTensor(np.array([y])))
    predict=model(x)
    print(predict)
    right = rightness(predict,y[0][1])
    rights.append(right)   
right_ratio=1.0*np.sum([i[0] for i in rights])/np.sum([i[1] for i in rights])


tensor([[4.8360]], grad_fn=<AddmmBackward>)
tensor([[4.8000]]) tensor(4.9000)
tensor([[4.6644]], grad_fn=<AddmmBackward>)
tensor([[4.7000]]) tensor(4.8000)
tensor([[4.7748]], grad_fn=<AddmmBackward>)
tensor([[4.8000]]) tensor(4.6000)
tensor([[4.7886]], grad_fn=<AddmmBackward>)
tensor([[4.8000]]) tensor(4.7000)
tensor([[4.7344]], grad_fn=<AddmmBackward>)
tensor([[4.7000]]) tensor(4.7000)
tensor([[4.7513]], grad_fn=<AddmmBackward>)
tensor([[4.8000]]) tensor(4.7000)
tensor([[4.7247]], grad_fn=<AddmmBackward>)
tensor([[4.7000]]) tensor(4.7000)
tensor([[4.7011]], grad_fn=<AddmmBackward>)
tensor([[4.7000]]) tensor(4.6000)
tensor([[4.7406]], grad_fn=<AddmmBackward>)
tensor([[4.7000]]) tensor(4.8000)
tensor([[4.7105]], grad_fn=<AddmmBackward>)
tensor([[4.7000]]) tensor(4.7000)
tensor([[4.7401]], grad_fn=<AddmmBackward>)
tensor([[4.7000]]) tensor(4.6000)
tensor([[4.7305]], grad_fn=<AddmmBackward>)
tensor([[4.7000]]) tensor(4.8000)
tensor([[4.7372]], grad_fn=<AddmmBackward>)
tensor([[4.7000]]) t

tensor([[4.7000]]) tensor(4.5000)
tensor([[4.6807]], grad_fn=<AddmmBackward>)
tensor([[4.7000]]) tensor(4.7000)
tensor([[4.7721]], grad_fn=<AddmmBackward>)
tensor([[4.8000]]) tensor(4.8000)
tensor([[4.7348]], grad_fn=<AddmmBackward>)
tensor([[4.7000]]) tensor(4.9000)
tensor([[4.8532]], grad_fn=<AddmmBackward>)
tensor([[4.9000]]) tensor(4.8000)
tensor([[4.7069]], grad_fn=<AddmmBackward>)
tensor([[4.7000]]) tensor(4.9000)
tensor([[4.7038]], grad_fn=<AddmmBackward>)
tensor([[4.7000]]) tensor(4.5000)
tensor([[4.7025]], grad_fn=<AddmmBackward>)
tensor([[4.7000]]) tensor(4.7000)
tensor([[4.7245]], grad_fn=<AddmmBackward>)
tensor([[4.7000]]) tensor(4.8000)
tensor([[4.7342]], grad_fn=<AddmmBackward>)
tensor([[4.7000]]) tensor(4.7000)
tensor([[4.6807]], grad_fn=<AddmmBackward>)
tensor([[4.7000]]) tensor(4.6000)
tensor([[4.5945]], grad_fn=<AddmmBackward>)
tensor([[4.6000]]) tensor(4.2000)
tensor([[4.7592]], grad_fn=<AddmmBackward>)
tensor([[4.8000]]) tensor(4.8000)
tensor([[4.6487]], grad_fn=<Ad

tensor([[4.8000]]) tensor(4.9000)
tensor([[4.6527]], grad_fn=<AddmmBackward>)
tensor([[4.7000]]) tensor(4.7000)
tensor([[4.7227]], grad_fn=<AddmmBackward>)
tensor([[4.7000]]) tensor(4.7000)
tensor([[4.7468]], grad_fn=<AddmmBackward>)
tensor([[4.7000]]) tensor(4.8000)
tensor([[4.7058]], grad_fn=<AddmmBackward>)
tensor([[4.7000]]) tensor(4.9000)
tensor([[4.6915]], grad_fn=<AddmmBackward>)
tensor([[4.7000]]) tensor(4.7000)
tensor([[4.7468]], grad_fn=<AddmmBackward>)
tensor([[4.7000]]) tensor(4.9000)
tensor([[4.7270]], grad_fn=<AddmmBackward>)
tensor([[4.7000]]) tensor(4.5000)
tensor([[4.8202]], grad_fn=<AddmmBackward>)
tensor([[4.8000]]) tensor(4.9000)
tensor([[4.7225]], grad_fn=<AddmmBackward>)
tensor([[4.7000]]) tensor(4.7000)
tensor([[4.6565]], grad_fn=<AddmmBackward>)
tensor([[4.7000]]) tensor(4.7000)
tensor([[4.7129]], grad_fn=<AddmmBackward>)
tensor([[4.7000]]) tensor(4.7000)
tensor([[4.7565]], grad_fn=<AddmmBackward>)
tensor([[4.8000]]) tensor(4.8000)
tensor([[4.6944]], grad_fn=<Ad

tensor([[4.7388]], grad_fn=<AddmmBackward>)
tensor([[4.7000]]) tensor(4.7000)
tensor([[4.7034]], grad_fn=<AddmmBackward>)
tensor([[4.7000]]) tensor(4.7000)
tensor([[4.7121]], grad_fn=<AddmmBackward>)
tensor([[4.7000]]) tensor(4.7000)
tensor([[4.7302]], grad_fn=<AddmmBackward>)
tensor([[4.7000]]) tensor(4.7000)
tensor([[4.7655]], grad_fn=<AddmmBackward>)
tensor([[4.8000]]) tensor(4.8000)
tensor([[4.7238]], grad_fn=<AddmmBackward>)
tensor([[4.7000]]) tensor(4.7000)
tensor([[4.7146]], grad_fn=<AddmmBackward>)
tensor([[4.7000]]) tensor(4.6000)
tensor([[4.7061]], grad_fn=<AddmmBackward>)
tensor([[4.7000]]) tensor(4.6000)
tensor([[4.7023]], grad_fn=<AddmmBackward>)
tensor([[4.7000]]) tensor(4.7000)
tensor([[4.7535]], grad_fn=<AddmmBackward>)
tensor([[4.8000]]) tensor(4.8000)
tensor([[4.8011]], grad_fn=<AddmmBackward>)
tensor([[4.8000]]) tensor(4.9000)
tensor([[4.6768]], grad_fn=<AddmmBackward>)
tensor([[4.7000]]) tensor(4.7000)
tensor([[4.7322]], grad_fn=<AddmmBackward>)
tensor([[4.7000]]) t

tensor([[4.7000]]) tensor(4.8000)
tensor([[4.7544]], grad_fn=<AddmmBackward>)
tensor([[4.8000]]) tensor(4.8000)
tensor([[4.7438]], grad_fn=<AddmmBackward>)
tensor([[4.7000]]) tensor(4.8000)
tensor([[4.7081]], grad_fn=<AddmmBackward>)
tensor([[4.7000]]) tensor(4.7000)
tensor([[4.7127]], grad_fn=<AddmmBackward>)
tensor([[4.7000]]) tensor(4.7000)
tensor([[4.7178]], grad_fn=<AddmmBackward>)
tensor([[4.7000]]) tensor(4.7000)
tensor([[4.6386]], grad_fn=<AddmmBackward>)
tensor([[4.6000]]) tensor(4.7000)
tensor([[4.7002]], grad_fn=<AddmmBackward>)
tensor([[4.7000]]) tensor(4.5000)
tensor([[4.7274]], grad_fn=<AddmmBackward>)
tensor([[4.7000]]) tensor(4.7000)
tensor([[4.7473]], grad_fn=<AddmmBackward>)
tensor([[4.7000]]) tensor(4.8000)
tensor([[4.7490]], grad_fn=<AddmmBackward>)
tensor([[4.7000]]) tensor(4.7000)
tensor([[4.7573]], grad_fn=<AddmmBackward>)
tensor([[4.8000]]) tensor(4.8000)
tensor([[4.7213]], grad_fn=<AddmmBackward>)
tensor([[4.7000]]) tensor(4.7000)
tensor([[4.6679]], grad_fn=<Ad

...

tensor([[4.7000]]) tensor(4.7000)
tensor([[4.7169]], grad_fn=<AddmmBackward>)
tensor([[4.7000]]) tensor(4.7000)
tensor([[4.7173]], grad_fn=<AddmmBackward>)
tensor([[4.7000]]) tensor(4.7000)
tensor([[4.7100]], grad_fn=<AddmmBackward>)
tensor([[4.7000]]) tensor(4.7000)
tensor([[4.6832]], grad_fn=<AddmmBackward>)
tensor([[4.7000]]) tensor(4.7000)
tensor([[4.6692]], grad_fn=<AddmmBackward>)
tensor([[4.7000]]) tensor(4.7000)
tensor([[4.7015]], grad_fn=<AddmmBackward>)
tensor([[4.7000]]) tensor(4.8000)
tensor([[4.7360]], grad_fn=<AddmmBackward>)
tensor([[4.7000]]) tensor(4.7000)
tensor([[4.7469]], grad_fn=<AddmmBackward>)
tensor([[4.7000]]) tensor(4.8000)
tensor([[4.7130]], grad_fn=<AddmmBackward>)
tensor([[4.7000]]) tensor(4.5000)
tensor([[4.7533]], grad_fn=<AddmmBackward>)
tensor([[4.8000]]) tensor(4.8000)
tensor([[4.7306]], grad_fn=<AddmmBackward>)
tensor([[4.7000]]) tensor(4.7000)
tensor([[4.7444]], grad_fn=<AddmmBackward>)
tensor([[4.7000]]) tensor(4.8000)
tensor([[4.7454]], grad_fn=<Ad

tensor([[4.7000]]) tensor(4.9000)
tensor([[4.7190]], grad_fn=<AddmmBackward>)
tensor([[4.7000]]) tensor(4.7000)
tensor([[4.7075]], grad_fn=<AddmmBackward>)
tensor([[4.7000]]) tensor(4.7000)
tensor([[4.6944]], grad_fn=<AddmmBackward>)
tensor([[4.7000]]) tensor(4.7000)
tensor([[4.7063]], grad_fn=<AddmmBackward>)
tensor([[4.7000]]) tensor(4.7000)
tensor([[4.6665]], grad_fn=<AddmmBackward>)
tensor([[4.7000]]) tensor(4.6000)
tensor([[4.7233]], grad_fn=<AddmmBackward>)
tensor([[4.7000]]) tensor(4.8000)
tensor([[4.6783]], grad_fn=<AddmmBackward>)
tensor([[4.7000]]) tensor(4.7000)
tensor([[4.6880]], grad_fn=<AddmmBackward>)
tensor([[4.7000]]) tensor(4.8000)
tensor([[4.6765]], grad_fn=<AddmmBackward>)
tensor([[4.7000]]) tensor(4.7000)
tensor([[4.6410]], grad_fn=<AddmmBackward>)
tensor([[4.6000]]) tensor(4.5000)
tensor([[4.6352]], grad_fn=<AddmmBackward>)
tensor([[4.6000]]) tensor(4.6000)
tensor([[4.7594]], grad_fn=<AddmmBackward>)
tensor([[4.8000]]) tensor(4.7000)
tensor([[4.7211]], grad_fn=<Ad

tensor([[4.7000]]) tensor(4.6000)
tensor([[4.7307]], grad_fn=<AddmmBackward>)
tensor([[4.7000]]) tensor(4.7000)
tensor([[4.7167]], grad_fn=<AddmmBackward>)
tensor([[4.7000]]) tensor(4.8000)
tensor([[4.7028]], grad_fn=<AddmmBackward>)
tensor([[4.7000]]) tensor(4.7000)
tensor([[4.7354]], grad_fn=<AddmmBackward>)
tensor([[4.7000]]) tensor(4.7000)
tensor([[4.7358]], grad_fn=<AddmmBackward>)
tensor([[4.7000]]) tensor(4.7000)
tensor([[4.6963]], grad_fn=<AddmmBackward>)
tensor([[4.7000]]) tensor(4.8000)
tensor([[4.8477]], grad_fn=<AddmmBackward>)
tensor([[4.8000]]) tensor(4.7000)
tensor([[4.8176]], grad_fn=<AddmmBackward>)
tensor([[4.8000]]) tensor(4.9000)
tensor([[4.7175]], grad_fn=<AddmmBackward>)
tensor([[4.7000]]) tensor(4.7000)
tensor([[4.7421]], grad_fn=<AddmmBackward>)
tensor([[4.7000]]) tensor(4.8000)
tensor([[4.7139]], grad_fn=<AddmmBackward>)
tensor([[4.7000]]) tensor(4.7000)
tensor([[4.7568]], grad_fn=<AddmmBackward>)
tensor([[4.8000]]) tensor(4.7000)
tensor([[4.7025]], grad_fn=<Ad

tensor([[4.7000]]) tensor(4.9000)
tensor([[4.6084]], grad_fn=<AddmmBackward>)
tensor([[4.6000]]) tensor(4.7000)
tensor([[4.7094]], grad_fn=<AddmmBackward>)
tensor([[4.7000]]) tensor(4.8000)
tensor([[4.7048]], grad_fn=<AddmmBackward>)
tensor([[4.7000]]) tensor(4.7000)
tensor([[4.7283]], grad_fn=<AddmmBackward>)
tensor([[4.7000]]) tensor(4.7000)
tensor([[4.7166]], grad_fn=<AddmmBackward>)
tensor([[4.7000]]) tensor(4.7000)
tensor([[4.7803]], grad_fn=<AddmmBackward>)
tensor([[4.8000]]) tensor(4.8000)
tensor([[4.7130]], grad_fn=<AddmmBackward>)
tensor([[4.7000]]) tensor(4.8000)
tensor([[4.7096]], grad_fn=<AddmmBackward>)
tensor([[4.7000]]) tensor(4.8000)
tensor([[4.6917]], grad_fn=<AddmmBackward>)
tensor([[4.7000]]) tensor(4.6000)
tensor([[4.6709]], grad_fn=<AddmmBackward>)
tensor([[4.7000]]) tensor(4.7000)
tensor([[4.7119]], grad_fn=<AddmmBackward>)
tensor([[4.7000]]) tensor(4.7000)
tensor([[4.7037]], grad_fn=<AddmmBackward>)
tensor([[4.7000]]) tensor(4.8000)
tensor([[4.7159]], grad_fn=<Ad

tensor([[4.7265]], grad_fn=<AddmmBackward>)
tensor([[4.7000]]) tensor(4.7000)
tensor([[4.7440]], grad_fn=<AddmmBackward>)
tensor([[4.7000]]) tensor(4.9000)
tensor([[4.6773]], grad_fn=<AddmmBackward>)
tensor([[4.7000]]) tensor(4.5000)
tensor([[4.7602]], grad_fn=<AddmmBackward>)
tensor([[4.8000]]) tensor(4.8000)
tensor([[4.7299]], grad_fn=<AddmmBackward>)
tensor([[4.7000]]) tensor(4.9000)
tensor([[4.8919]], grad_fn=<AddmmBackward>)
tensor([[4.9000]]) tensor(4.6000)
tensor([[4.7050]], grad_fn=<AddmmBackward>)
tensor([[4.7000]]) tensor(4.7000)
tensor([[4.7451]], grad_fn=<AddmmBackward>)
tensor([[4.7000]]) tensor(4.9000)
tensor([[4.7398]], grad_fn=<AddmmBackward>)
tensor([[4.7000]]) tensor(4.5000)
tensor([[4.6833]], grad_fn=<AddmmBackward>)
tensor([[4.7000]]) tensor(4.6000)
tensor([[4.7243]], grad_fn=<AddmmBackward>)
tensor([[4.7000]]) tensor(4.7000)
tensor([[4.7357]], grad_fn=<AddmmBackward>)
tensor([[4.7000]]) tensor(4.8000)
tensor([[4.7803]], grad_fn=<AddmmBackward>)
tensor([[4.8000]]) t

In [26]:
y

tensor([[4.8000, 4.7000, 4.8000, 4.7000, 4.8000, 4.0000]])

In [36]:
# 损失函数为交叉熵
MSE = t.nn.MSELoss()
optimizer = optim.SGD(model.parameters(),lr=0.001)
records=[]

losses=[]
val_losses=[]
for epoch in range(5):
    for i,data in enumerate(zip(train_data,train_label)):
        x,y=data
        x=Variable(t.FloatTensor(x).view(1,-1))
        y=Variable(t.FloatTensor(np.array([y])))
        predict=model(x)
        loss = MSE(predict,y[0][1])
        losses.append(loss.data.numpy())
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if i%1000==0:
            
            rights=[]
            for j,val in enumerate(zip(valid_data,valid_label)):
                x,y = val
                x=Variable(t.FloatTensor(x).view(1,-1))
                y=Variable(t.FloatTensor(np.array([y])))
                predict = model(x)
                right = rightness(predict,y[0][1])
                rights.append(right)
                loss = MSE(predict,y[0][1])
                val_losses.append(loss.data.numpy())
            
            right_ratio=1.0*np.sum([i[0] for i in rights])/np.sum([i[1] for i in rights])
            print('第{}轮，训练损失： {:.6f}，校验损失：{:.6f}，校验准确度：{:.2f}'.format(epoch,np.mean(losses),np.mean(val_losses),right_ratio))
            records.append([np.mean(losses),np.mean(val_losses),right_ratio])

第0轮，训练损失： 24.255213，校验损失：22.495483，校验准确度：0.00
第0轮，训练损失： 3.052257，校验损失：11.292666，校验准确度：0.16
第0轮，训练损失： 1.590247，校验损失：7.552522，校验准确度：0.20
第0轮，训练损失： 1.076281，校验损失：5.672370，校验准确度：0.29
第0轮，训练损失： 0.816070，校验损失：4.543293，校验准确度：0.31
第0轮，训练损失： 0.657515，校验损失：3.789811，校验准确度：0.33
第0轮，训练损失： 0.551408，校验损失：3.251297，校验准确度：0.36
第0轮，训练损失： 0.475331，校验损失：2.847061，校验准确度：0.37
第0轮，训练损失： 0.418359，校验损失：2.532534，校验准确度：0.38
第0轮，训练损失： 0.374334，校验损失：2.280877，校验准确度：0.40
第0轮，训练损失： 0.338504，校验损失：2.074906，校验准确度：0.40
第0轮，训练损失： 0.309160，校验损失：1.903288，校验准确度：0.39
第0轮，训练损失： 0.284892，校验损失：1.758038，校验准确度：0.41
第0轮，训练损失： 0.264152，校验损失：1.633498，校验准确度：0.41
第0轮，训练损失： 0.246276，校验损失：1.525559，校验准确度：0.41
第0轮，训练损失： 0.230685，校验损失：1.431096，校验准确度：0.42
第0轮，训练损失： 0.217225，校验损失：1.347748，校验准确度：0.41
第0轮，训练损失： 0.205384，校验损失：1.273678，校验准确度：0.41
第0轮，训练损失： 0.194862，校验损失：1.207403，校验准确度：0.41
第0轮，训练损失： 0.185664，校验损失：1.147770，校验准确度：0.41
第0轮，训练损失： 0.177101，校验损失：1.093803，校验准确度：0.41
第1轮，训练损失： 0.175988，校验损失：1.044735，校验准确度：0.41
第1轮，训练损失： 0.168767，校验损失：0.999

In [ ]:
for i,data in enumerate(zip(test_data,test_label)):
    x,y=data
    x=Variable(t.FloatTensor(x).view(1,-1))
    y=Variable(t.FloatTensor(np.array([y])))
    predict = model(x)
    right = rightness(predict,y)
    np.round(predict.data,1)
    print(np.round(predict.data,1))
    print(y)
    print('----------------------------------------------------')